## install & load packages

In [1]:
!pip -q install git+https://github.com/mwshinn/PyDDM
import pyddm
import pyddm.plot
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pyddm import Sample

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## prep data

# 12/weighted drifts
### assuming equal effect of congruent & incongruent 80% cues

In [3]:
def drift_weights(t, trueCue, trueCongruence, coherence, signal1_onset, noise2_onset, signal2_onset,
                      m_noise1, m50_noise1, m_signal1, m50_signal1, v_signal1, v50_signal1,
                      m_noise2, m50_noise2, m_signal2, m50_signal2, v_signal2, v50_signal2):
    if t < signal1_onset:
        return trueCue * (m_noise1 if trueCue > 0.5 else m50_noise1)
    elif t < noise2_onset:
        if trueCongruence == 'congruent' or 'neutral':
            return trueCue * m_signal1 + coherence * (v_signal1 if trueCue > 0.5 else v50_signal1)
        else:
            return -trueCue * m_signal1 + coherence * v_signal1
    elif t < signal2_onset:
        if trueCongruence == 'congruent' or 'neutral':
            return trueCue * (m_noise2 if trueCue > 0.5 else m50_noise2)
        else:
            return -trueCue * m_noise2
    else:
        if trueCongruence == 'congruent' or 'neutral':
            return trueCue * (m_signal2 if trueCue > 0.5 else m50_signal2) + coherence * (v_signal2 if trueCue > 0.8 else v50_signal2)
        else:
            return -trueCue * m_signal2 + coherence * v_signal2

def drift_weights2(t, congCue, coherence, signal1_onset, noise2_onset, signal2_onset,
                      m_noise1, m50_noise1, m_signal1, m50_signal1, v_signal1, v50_signal1,
                      m_noise2, m50_noise2, m_signal2, m50_signal2, v_signal2, v50_signal2):
    if t < signal1_onset:
        return congCue * (m_noise1 if abs(congCue) > 0.5 else m50_noise1)
    elif t < noise2_onset:
        return congCue * (m_signal1 if abs(congCue) > 0.5 else m50_signal1) + coherence * (v_signal1 if abs(congCue) > 0.5 else v50_signal1)
    elif t < signal2_onset:
        return congCue * (m_noise2 if abs(congCue) > 0.5 else m50_noise2)
    else:
        return congCue * (m_signal2 if abs(congCue) > 0.5 else m50_signal2) + coherence * (v_signal2 if abs(congCue) > 0.5 else v50_signal2)



# specify model
model = pyddm.gddm(
    drift = drift_weights2,
     starting_position = 0,
     bound="B",
     T_dur = 4.1,
     nondecision='ndt',
     parameters={'B': (0.5, 25), 'ndt': (0.01, 1.5),
                    'm_noise1': (0, 10), 'm50_noise1': (0,10), 'm_signal1': (0,10),
                    'm50_signal1': (0, 10), 'v_signal1': (0, 10), 'v50_signal1': (0, 10),
                    'm_noise2': (0, 10), 'm50_noise2': (0,10), 'm_signal2': (0,10),
                    'm50_signal2': (0, 10), 'v_signal2': (0, 10), 'v50_signal2': (0,10)},
    conditions = ['congCue', 'trueCongruence', 'coherence', 'signal1_onset', 'noise2_onset', 'signal2_onset'])

pyddm.plot.model_gui_jupyter(model, conditions = ['congCue': 0,], data_dt = 0.05)

AssertionError: Please pass all conditions needed by the model in the 'conditions' argument.

In [7]:
df = pd.read_csv('inference_all.csv')
sample_65 = pyddm.Sample.from_pandas_dataframe(df[df['subID']==65], rt_column_name="RT", choice_column_name="accuracy")

pyddm.plot.model_gui_jupyter(model, sample_65, data_dt = 0.05)

Output()